In [1]:
# Parameters
kms_key = "arn:aws:kms:us-west-2:000000000000:1234abcd-12ab-34cd-56ef-1234567890ab"


# Training a Tensorflow Model on MNIST

MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). This tutorial will show how to train a Tensorflow V2 model on MNIST model on SageMaker.


In [2]:
import os
import json

import sagemaker
from sagemaker.tensorflow import TensorFlow
from sagemaker import get_execution_role

sess = sagemaker.Session()

role = get_execution_role()

output_path = "s3://" + sess.default_bucket() + "/tensorflow/mnist"

## TensorFlow Estimator

The `TensorFlow` class allows you to run your training script on SageMaker
infrastracture in a containerized environment. In this notebook, we
refer to this container as *training container*. 

You need to configure
it with the following parameters to set up the environment:

- entry_point: A user defined python file to be used by the training container as the 
instructions for training. We will further discuss this file in the next subsection

- role: An IAM role to make AWS service requests

- instance_type: The type of SageMaker instance to run your training script. 
Set it to `local` if you want to run the training job on 
the SageMaker instance you are using to run this notebook

- model_dir: S3 bucket URI where the checkpoint data and models can be exported to during training (default: None). 
To disable having model_dir passed to your training script, set `model_dir`=False

- instance count: The number of instances you need to run your training job. 
Multiple instances are needed for distributed training

- output_path: 
S3 bucket URI to save training output (model artifacts and output files)

- framework_version: The version of TensorFlow you need to use.

- py_version: The python version you need to use

For more information, see [the API reference](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase)



## Implement the entry point for training

The entry point for training is a python script that provides all 
the code for training a TensorFlow model. It is used by the SageMaker 
TensorFlow Estimator (`TensorFlow` class above) as the entry point for running the training job.

Under the hood, SageMaker TensorFlow Estimator downloads a docker image
with runtime environemnts 
specified by the parameters you used to initiated the
estimator class and it injects the training script into the 
docker image to be used as the entry point to run the container.

In the rest of the notebook, we use *training image* to refer to the 
docker image specified by the TensorFlow Estimator and *training container*
to refer to the container that runs the training image. 

This means your training script is very similar to a training script
you might run outside Amazon SageMaker, but it can access the useful environment 
variables provided by the training image. Checkout [the short list of environment variables provided by the SageMaker service](https://sagemaker.readthedocs.io/en/stable/frameworks/mxnet/using_mxnet.html?highlight=entry%20point) to see some common environment 
variables you might used. Checkout [the complete list of environment variables](https://github.com/aws/sagemaker-training-toolkit/blob/master/ENVIRONMENT_VARIABLES.md) for a complete 
description of all environment variables your training script
can access to. 

In this example, we use the training script `code/train.py`
as the entry point for our TensorFlow Estimator. 

In [3]:
!pygmentize 'code/train.py'

from __future__ import print_function

import argparse
import gzip
import json
import logging
import os
import traceback

import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, Dense, Flatten

logging.basicConfig(level=logging.DEBUG)

# Define the model object


class SmallConv(Model):
    def __init__(self):
        super(SmallConv, self).__init__()
        self.conv1 = Conv2D(32, 3, activation="relu")
        self.flatten = Flatten()
        self.d1 = Dense(128, activation="relu")
        self.d2 = Dense(10)

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)


# Decode and preprocess data
def convert_to_numpy(data_dir, images_file, labels_file):
    """Byte string to numpy arrays"""
    with gzip.open(os.path.join(data_dir, images_file), "rb") as f:
        images = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1, 28, 28)

    with gzi

, keepdims=True)
    # avoid division by zero
    std = np.std(x, axis=axis, keepdims=True) + eps
    return (x - mean) / std


# Training logic


def train(args):
    # create data loader from the train / test channels
    x_train, y_train = mnist_to_numpy(data_dir=args.train, train=True)
    x_test, y_test = mnist_to_numpy(data_dir=args.test, train=False)

    x_train, x_test = x_train.astype(np.float32), x_test.astype(np.float32)

    # normalize the inputs to mean 0 and std 1
    x_train, x_test = normalize(x_train, (1, 2)), normalize(x_test, (1, 2))

    # expand channel axis
    # tf uses depth minor convention
    x_train, x_test = np.expand_dims(x_train, axis=3), np.expand_dims(x_test, axis=3)

    # normalize the data to mean 0 and std 1
    train_loader = (
        tf.data.Dataset.from_tensor_slices((x_train, y_train))
        .shuffle(len(x_train))
        .batch(args.batch_size)
    )

    test_loader = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(args.batch_s

### Set hyperparameters

In addition, TensorFlow estimator allows you to parse command line arguments
to your training script via `hyperparameters`.

<span style="color:red"> Note: local mode is not supported in SageMaker Studio. </span>

In [4]:
# set local_mode to be True if you want to run the training script
# on the machine that runs this notebook

local_mode = False

if local_mode:
    instance_type = "local"
else:
    instance_type = "ml.c4.xlarge"

est = TensorFlow(
    entry_point="train.py",
    source_dir="code",  # directory of your training script
    role=role,
    framework_version="2.3.1",
    model_dir=False,  # don't pass --model_dir to your training script
    py_version="py37",
    instance_type=instance_type,
    instance_count=1,
    volume_size=250,
    output_path=output_path,
    hyperparameters={
        "batch-size": 512,
        "epochs": 1,
        "learning-rate": 1e-3,
        "beta_1": 0.9,
        "beta_2": 0.999,
    },
)

The training container executes your training script like

```
python train.py --batch-size 32 --epochs 1 --learning-rate 0.001
    --beta_1 0.9 --beta_2 0.999
```

## Set up channels for training and testing data

You need to tell `TensorFlow` estimator where to find your training and 
testing data. It can be a link to an S3 bucket or it can be a path
in your local file system if you use local mode. In this example,
we download the MNIST data from a public S3 bucket and upload it 
to your default bucket. 

In [5]:
import logging
import boto3
from botocore.exceptions import ClientError

# Download training and testing data from a public S3 bucket


def download_from_s3(data_dir="/tmp/data", train=True):
    """Download MNIST dataset and convert it to numpy array

    Args:
        data_dir (str): directory to save the data
        train (bool): download training set

    Returns:
        None
    """

    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    if train:
        images_file = "train-images-idx3-ubyte.gz"
        labels_file = "train-labels-idx1-ubyte.gz"
    else:
        images_file = "t10k-images-idx3-ubyte.gz"
        labels_file = "t10k-labels-idx1-ubyte.gz"

    # download objects
    s3 = boto3.client("s3")
    bucket = f"sagemaker-sample-files"
    for obj in [images_file, labels_file]:
        key = os.path.join("datasets/image/MNIST", obj)
        dest = os.path.join(data_dir, obj)
        if not os.path.exists(dest):
            s3.download_file(bucket, key, dest)
    return


download_from_s3("/tmp/data", True)
download_from_s3("/tmp/data", False)

In [6]:
# upload to the default bucket

prefix = "mnist"
bucket = sess.default_bucket()
loc = sess.upload_data(path="/tmp/data", bucket=bucket, key_prefix=prefix)

channels = {"training": loc, "testing": loc}

The keys of the dictionary `channels` are parsed to the training image
and it creates the environment variable `SM_CHANNEL_<key name>`. 

In this example, `SM_CHANNEL_TRAINING` and `SM_CHANNEL_TESTING` are created in the training image (checkout 
how `code/train.py` access these variables). For more information,
see: [SM_CHANNEL_{channel_name}](https://github.com/aws/sagemaker-training-toolkit/blob/master/ENVIRONMENT_VARIABLES.md#sm_channel_channel_name)

If you want, you can create a channel for validation:
```
channels = {
    'training': train_data_loc,
    'validation': val_data_loc,
    'test': test_data_loc
    }
```
You can then access this channel within your training script via
`SM_CHANNEL_VALIDATION`.

## Run the training script on SageMaker
Now, the training container has everything to execute your training
script. You can start the container by calling `fit` method.

In [7]:
est.fit(inputs=channels)

2022-03-24 00:12:30 Starting - Starting the training job.

.

.


2022-03-24 00:12:48 Starting - Preparing the instances for trainingProfilerReport-1648080750: InProgress
.

.

...


2022-03-24 00:55:55 Training - Training image download completed. Training in progress..

2022-03-24 00:55:52.415362: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2022-03-24 00:55:52.432789: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2022-03-24 00:55:53.069478: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2022-03-24 00:55:59,293 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2022-03-24 00:55:59,301 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-03-24 00:55:59,999 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-03-24 00:56:00,019 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-03-24 00:56:00,038 sagemaker-training-toolkit INFO     No GPUs detected (normal if

Epoch 1, Loss: 0.2663554549217224, Accuracy: 91.77333068847656, Test Loss: 0.11055289208889008, Test Accuracy: 96.87999725341797
2022-03-24 00:56:00.490148: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2022-03-24 00:56:00.490370: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2022-03-24 00:56:00.534650: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should


2022-03-24 00:56:55 Uploading - Uploading generated training model
2022-03-24 00:56:55 Completed - Training job completed
ProfilerReport-1648080750: NoIssuesFound


Training seconds: 2526
Billable seconds: 2526


## Inspect and store model data

Now, the training is finished, the model artifact has been saved in 
the `output_path`. We 

In [8]:
tf_mnist_model_data = est.model_data
print("Model artifact saved at:\n", tf_mnist_model_data)

Model artifact saved at:
 s3://sagemaker-us-west-2-000000000000/tensorflow/mnist/tensorflow-training-2022-03-24-00-12-30-135/output/model.tar.gz


We will store the variable `model_data` in the current notebook kernel. 
In the [next notebook](get_started_with_mnist_deploy.ipynb), you will learn how to retrieve the model artifact and deploy to a SageMaker
endpoint.

In [9]:
%store tf_mnist_model_data

Stored 'tf_mnist_model_data' (str)


## Test and debug the entry point before executing the training container

The entry point `code/train.py` provided here has been tested and it can be executed in the training container. 
When you develop your own training script, it is a good practice to simulate the container environment 
in the local shell and test it before sending it to SageMaker, because debugging in a containerized environment
is rather cumbersome. The following script shows how you can test your training script:

In [10]:
!pygmentize code/test_train.py

import json
import os
import sys

import boto3
from train import parse_args, train

dirname = os.path.dirname(os.path.abspath(__file__))

with open(os.path.join(dirname, "config.json"), "r") as f:
    CONFIG = json.load(f)


def download_from_s3(data_dir="/tmp/data", train=True):
    """Download MNIST dataset and convert it to numpy array
    Args:
        data_dir (str): directory to save the data
        train (bool): download training set
    Returns:
        tuple of images and labels as numpy arrays
    """

    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    if train:
        images_file = "train-images-idx3-ubyte.gz"
        labels_file = "train-labels-idx1-ubyte.gz"
    else:
        images_file = "t10k-images-idx3-ubyte.gz"
        labels_file = "t10k-labels-idx1-ubyte.gz"

    # download objects
    s3 = boto3.client("s3")
    bucket = CONFIG["public_bucket"]
    for obj in [images_file, labels_file]:
        key = os.path.join("datasets/image/MNIST", obj)

In [the next notebook](get_started_mnist_deploy.ipynb) you will see how to deploy your 
trained model artifacts to a SageMaker endpoint. 